## Quantifying results from Google query and using logistic regression to predict likelyhood of VIP

Will need to create a training dataset. Could integrate ChatGPT using snippets or websites from the following promt: "Using the following snippets. Give a rating from 0-100 on the likeliness that the person is famous. Respond only with the rating. snippets: "

In [2]:
#import necessary libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
import os
import json
import instaloader
import requests

#load evn variables
API_KEY = os.environ['GOOGLE_API_KEY']
SEARCH_ENGINE_ID = os.environ['SEARCH_ENGINE_ID']

# Functions to Quantify Google query results

In [4]:
#define functions


#google name and output json file
#takes in name api_key, search_engine_id
def search_name(name, api_key, search_engine_id):   
    # Iterate over the names and grab search results using Google Custom Search API
    search_url = f'https://www.googleapis.com/customsearch/v1?q={name}&cx={search_engine_id}&key={api_key}'
    response = requests.get(search_url)
    data = json.loads(response.text)
    return data


# checks email domain for company emails, 
# input takes in dataframe object
# returns dataframe with 1 or 0 for check email column 
def check_email(df):
    xlst = ['graduatehotels.com','schultehospitality.com','schultedc.com']
    lst = []
    for i in range(len(df)):
        if df.loc[i] == "nan":
            lst.append(0)
        else:
            if df.loc[i].split("@")[1].lower() in xlst:
                lst.append(1)
            else:
                lst.append(0)
    return lst


#clean data set to add full names and split email for domain, 
# takes in string file path
# return dataframe object
def clean_data(file_path):
    df = pd.read_excel(file_path)
    df['names'] = df['First Name'] + " " + df['Last Name']
    df['Email'].fillna("nan",inplace=True)
    lenOfDF = [0 for i in range(len(df))]
    emails = df['Email']
    df['Email Check'] = check_email(emails)
    df['Websites'] = lenOfDF
    df['Social Media'] = lenOfDF
    df['keywords'] = lenOfDF
    return df


# instagram follower count
# takes in url of account from json['items'][int]['metatags']['og:url']
#  returns int follower count,
# WILL NEED TWEEPY FOR TWITTER FOLLOWER COUNT, DO NOT HAVE ACCESS YET
def instaCount(link):
    bot = instaloader.Instaloader()
    profile = instaloader.Profile.from_username(bot.context,link[26:-1])
    return profile.followers


# scans json for websites associated with name 
# takes in jsonObj and scans through for specific links
# links will be weighted using ints will need to tune weights later
# returns wighted score 
def scan_websites(jsonObj,firstName,lastName):
    WEBSITES = {'en.wikipedia.org':9,'www.forbes.com':10,'www.bloomberg.com':9,'www.imdb.com':9,'sanfran.com':3,'www.architecturaldigest.com':5,
                'www.tvinsider.com':5,'nypost.com':8,'www.themoviedb.org':7,'www.celebritynetworth.com':6,'www.tvguide.com':6,
                'ew.com':6,'www.thefamouspeople.com':8,'www.newyorker.com':8,'www.britannica.com':8,'www.investopedia.com':5,'abc.com':10,
                'www.cnbc.com':10,'www.nbc.com':10,'www.pbs.org':10,'www.espn.com':10,'www.biography.com':8,'www.newsweek.com':6,'www.cbsnews.com':10} # add the rest of links
    SOCIAL = ['www.instagram.com']
    score = 0
    k = 10 #only checks first 10, change for more, can do len for full
    first = firstName.lower()
    last = lastName.lower()
    social = False
    socInd = []
    if 'items' in jsonObj:
        for j in range(0,k): #only checks first 10
            if jsonObj['items'][j]['displayLink'] in WEBSITES:
                score += WEBSITES[jsonObj['items'][j]['displayLink']]
            if first in jsonObj['items'][j]['displayLink'].lower() or last in jsonObj['items'][j]['displayLink'].lower():
                score += 10 # adjust for weight
            if jsonObj['items'][j]['displayLink'] in SOCIAL:
                social = True
                socInd.append(j)
    return score ,social,socInd


#scans for keywords in snippet 
#will take in jsonObj and look for key words in the snippet
#could also weigh words 
#can also combine with scan websites 
#outputs int score
def scan_keywords(jsonObj):
    KEYWORDSW = {'owner':10,'died':-100_000_000,'dead':-100_000_000,'passed':-100_000_000} #add more words. CAN ADD NEGATIVE FOR DIED/DEATH/PASSING
    KEYWORDS = ['onwer','actress','owner','entrepenuer','CEO','NBA','NFL','MLB','boradcaster','founder','president'] #add more words
    k = 10 #only checks first 10, change for more, can do len for full
    score = 0
    if 'items' in jsonObj:
        for j in range(0,k): #only checks first 10
            for x in jsonObj['items'][j]['snippet'].split(" "):
                if x.lower() in KEYWORDS:
                    score +=1 #comment code for dictionary
                    # score += KEYWORDSW[x.lower()] for dictoinary
    return score

#scans json for websites and keywords. combines the 2 functions above
def scan_combined(jsonObj):
    WEBSITES = {'en.wikipedia.org':10} # add the rest of links. CAN INCLUDE NEGATIVE NUMBER WEBSITES (IE. ONION, SNOPS, SATIRE WEBSITES)
    KEYWORDS = ['owner'] #{owner:10} #uncomment for dictionary
    score = 0
    k = 10 #only checks first 10, change for more, can do len for full
    name = jsonObj[0]['queries']['request'][0]['searchTerms'].split(" ")
    if 'items' in jsonObj:
        for j in range(0,k): #only checks first 10
            if jsonObj['items'][j]['displayLink'] in WEBSITES:
                score += WEBSITES[jsonObj['items'][j]['displayLink']]
            if name[0] in jsonObj['items'][j]['displayLink'] or name[1] in jsonObj['items'][j]['displayLink']:
                score += 15 # adjust for weight
            for x in jsonObj['items'][j]['snippet'].split(" "):
                if x.lower() in KEYWORDS:
                    score += 10 #comment code for dictionary
                    #score += KEYWORDS[x.lower()] #uncomment for dic
    return score


def search_analyize(df):
    for i in range(len(df)):
        social = False
        data = search_name(df['names'].loc[i],API_KEY,SEARCH_ENGINE_ID)
        df.at[i,'Website'], social,inde = scan_websites(data,df['First Name'].loc[i],df['Last Name'].loc[i])
        df.at[i,'keywords'] = scan_keywords(data)
        if social:
            df.at[i,'Social Media'] = instaCount(data['items'][inde]['metatags']['og:url'])
    return df

## Test function

In [ ]:
test = clean_data('test file.xlsx')

jsondata = []
for i in range(len(test)):
    jsondata.append(str(search_name(test['names'].loc[i],API_KEY,SEARCH_ENGINE_ID)))
jsondata

In [25]:
x = search_name('jack',API_KEY,SEARCH_ENGINE_ID)
type(x)

dict

In [16]:
tt = test['data'].loc[0]
print(type(tt))

<class 'str'>


In [22]:
print(type(jsondata))
type(jsondata[0])

<class 'list'>


str

In [73]:
print(instaCount('https://www.instagram.com/mcuban/'))

1970777


# Calculating Likliness of VIP

## Default 
Will assume people with certain email domains to be considered VIP. 
Any Sales/Marketing VIPs will also be considered VIP by Default. 
With HMS we can filter them out by tagging guest profile as VIP

## Methodology
Using Logistic Regression trained on excel file containing names we know are VIPS and not VIPS. Will run the search function and analyize search data. Using the quantified results from analysis we will train our Logistic Regression model. This way we can hope to reduce false positives. 

In [ ]:
trainData = pd.read('traindata.xlsx')
df = clean_data(trainData)
df = search_analyize(df)

In [ ]:
y,X = df['famous'],df.drop('famous',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

model = LogisticRegression()

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

accuracy = model.score(X_test,y_test)

print(accuracy)